In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve#,scorer
import re
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [2]:
df = pd.read_csv('./data/Corona_NLP_test.csv', encoding='ISO-8859-1')


In [21]:
Sentiment_dictionary = {'Neutral': 0, 'Positive': 1,
                        'Extremely Negative': 2, 'Negative': 3,
                        'Extremely Positive': 4}
y_test = df['Sentiment'].apply(lambda x: Sentiment_dictionary[x])
y_test.head()


0    2
1    1
2    4
3    3
4    0
Name: Sentiment, dtype: int64

In [4]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [8]:
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt
df['Tweet'] = np.vectorize(remove_pattern)(df['OriginalTweet'], '@[\w]*')
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')
df['Tweet'] = [stemmer.lemmatize(word) for word in df['Tweet']]
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
df['Tweet'] = df['Tweet'].apply(lambda x: x.split())
df['Tweet'] = df['Tweet'].apply(lambda x: ''.join(w+" " for w in x))

C:\Users\Home\AppData\Local\Temp/ipykernel_15244/2242526795.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')


In [9]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,TRENDING New Yorkers encounter empty supermark...
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,When couldn find hand sanitizer Fred Meyer tur...
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,Find out how you can protect yourself and love...
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,#Panic buying hits #NewYork City anxious shopp...
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,#toiletpaper #dunnypaper #coronavirus #coronav...


In [10]:
X_test = df

In [11]:
X_test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,TRENDING New Yorkers encounter empty supermark...
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,When couldn find hand sanitizer Fred Meyer tur...
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,Find out how you can protect yourself and love...
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,#Panic buying hits #NewYork City anxious shopp...
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,#toiletpaper #dunnypaper #coronavirus #coronav...


In [25]:
with open("SGDClassifier_pipeline.dill", 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [26]:
predictions = pipeline.predict(X_test)
pd.DataFrame({'preds': predictions}).to_csv("test_predictions.csv", index=None)

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report( predictions, y_test))
#print(confusion_matrix(y_test, predictions))


              precision    recall  f1-score   support

           0       0.76      0.52      0.62       907
           1       0.31      0.52      0.39       568
           2       0.68      0.51      0.58       792
           3       0.33      0.51      0.40       664
           4       0.78      0.54      0.64       867

    accuracy                           0.52      3798
   macro avg       0.57      0.52      0.52      3798
weighted avg       0.61      0.52      0.54      3798

